In [1]:
from sklearn.datasets import load_iris
from sklearn import tree
from matplotlib import pyplot as plt
import numpy as np
import utils
iris = load_iris()
X, y = iris.data, iris.target
clf = tree.DecisionTreeClassifier(max_depth=8)
clf = clf.fit(X, y)
# clf.score(X, y)
sp, table = utils.find_sparsity(clf, X)
print(sp)
print(table)

0.2962962962962963
[[0 0 0 1]
 [0 0 1 1]
 [0 0 1 1]
 [0 0 1 1]
 [0 0 1 1]
 [0 0 1 1]
 [0 1 1 1]
 [0 1 1 1]
 [0 0 1 1]]


In [26]:
def find_sparsity(DT: tree.DecisionTreeClassifier, X: np.ndarray):
    table = map_CAM(DT, X)
    empty = 0
    for i in range(len(table[0])):
        if sum(table[:,i]) == 0:
            empty += 1
    return 1 - ((table == 1).sum() / (table.size - empty * len(table))), table

def map_CAM(DT: tree.DecisionTreeClassifier, X: np.ndarray):
    flag = False
    tree_text = tree.export_text(DT)
    lines = tree_text.split("\n")
    depth = DT.get_depth()
    table = []
    visited = []
    dog = []
    catch = []
    for l in lines[:-1]:
        app = depth - (l.find("feature") - 1)//4
        if app == depth + 1:
            dog.append(-1)
            this_line = [0 for _ in range(DT.n_features_in_)]
            for index, this_depth in visited:
                this_line[index] = 1
            # print(visited)
            if len(visited) != 0:
                end = visited[-1][1]
            if flag:
                end = catch[-1][1]
            for i in range(end, depth):
                for j in catch[::-1]:
                    if j[1] == i:
                        this_line[j[0]] = 1
            table.append(this_line)
        else:
            if flag:
                flag = False
            left = l.find("_") + 1
            right = l.find("<=") if l.find("<=") != -1 else l.find(">")
            index = int(l[left:right])
            if len(visited) == 0:
                visited.append((index, app))
            else:
                if visited[-1] != (index,app):
                    visited.append((index, app))
                else:
                    poped = visited.pop()
                    catch.append(poped)
                    flag = True
    table = np.array(table)
    return (table > 0).astype(np.int)
sp, table = find_sparsity(clf, X)
print(sp)
print(table)

0.2222222222222222
[[0 0 1 0]
 [0 0 1 1]
 [0 0 1 1]
 [0 0 1 1]
 [1 0 1 1]
 [1 0 1 1]
 [1 0 1 1]
 [1 0 1 1]
 [0 0 1 1]]


In [2]:
ax = plt.subplot()
tree.plot_tree(clf)
plt.savefig("lol.svg")
print(tree.export_text(clf))

|--- feature_3 <= 0.80
|   |--- class: 0
|--- feature_3 >  0.80
|   |--- feature_3 <= 1.75
|   |   |--- feature_2 <= 4.95
|   |   |   |--- feature_3 <= 1.65
|   |   |   |   |--- class: 1
|   |   |   |--- feature_3 >  1.65
|   |   |   |   |--- class: 2
|   |   |--- feature_2 >  4.95
|   |   |   |--- feature_3 <= 1.55
|   |   |   |   |--- class: 2
|   |   |   |--- feature_3 >  1.55
|   |   |   |   |--- feature_2 <= 5.45
|   |   |   |   |   |--- class: 1
|   |   |   |   |--- feature_2 >  5.45
|   |   |   |   |   |--- class: 2
|   |--- feature_3 >  1.75
|   |   |--- feature_2 <= 4.85
|   |   |   |--- feature_1 <= 3.10
|   |   |   |   |--- class: 2
|   |   |   |--- feature_1 >  3.10
|   |   |   |   |--- class: 1
|   |   |--- feature_2 >  4.85
|   |   |   |--- class: 2

